In [ ]:
!pip install spacy_transformers
# !pip install jedi fastapi kaleido python-multipart uvicorn
# !pip install -U pip setuptools wheel
!pip install spacy==3.6.0




In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json


In [3]:
spacy.__version__

'3.6.0'

In [4]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [5]:
! git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 12.34 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [6]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json'))
len(cv_data)

200

In [7]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

2023-12-04 09:09:28.361287: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 09:09:28.361358: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 09:09:28.361402: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 09:09:31.302872: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./d

In [13]:
cv_data[10]

["Kartik Sharma Systems Engineer - Infosys Ltd  Delhi, Delhi - Email me on Indeed: indeed.com/r/Kartik-Sharma/cc7951fd7809f35e  ● Qualified B.Tech in Information Technology with 2.5 years overall and 2 years' experience in SAP Security, Project Management and Software Support. ● Currently spearheading as Senior Systems Engineer with Infosys Ltd Pune, well versed in Analysis, Test and Support activities. ● Proficient in handling various projects and managing project risks. Possess up to date knowledge of latest technological advancements, regulations and statutory compliances in the industry. ● Instrumental in building relations with upper level decision makers, seizing control of critical problem areas and delivering on client commitments.  PROJECT ANNEXURE:  Project Name: RB (Reckitt Benckiser) Duration: Since April '16 Role: SAP Security Consultant Responsibilities: ● Technical analyst for sap security in production and non-production environments. ● Worked with Security related tabl

In [9]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    docbin = DocBin()

    for text, annot in tqdm(data):
        doc = nlp(text)
        annot = annot['entities']

        ents = []
        entity_indices = set()

        for start, end, label in annot:
            if start >= end:
                # Skip invalid annotation with start >= end
                continue

            overlapping_indices = set(range(start, end)) & entity_indices

            if overlapping_indices:
                # Skip only the overlapping indices in this annotation
                entity_indices -= overlapping_indices
            else:
                # Process non-overlapping indices
                entity_indices.update(range(start, end))

                try:
                    span = doc.char_span(start, end, label=label, alignment_mode='strict')
                except Exception as e:
                    file.write(f"{[start, end]}    {text}\n")
                    print(f"Error processing span: {[start, end]}, {e}")
                    continue

                if span is not None:
                    ents.append(span)

        try:
            doc.ents = ents
            docbin.add(doc)
        except Exception as e:
            print(f"Error adding Doc to DocBin: {e}")
            pass

    return docbin

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size= 0.3)

In [11]:
len(train), len(test)

(140, 60)

In [12]:
with open('error.txt', 'w') as file:
    train_docbin = get_spacy_doc(file, train)
    train_docbin.to_disk('train_data.spacy')

    test_docbin = get_spacy_doc(file, test)
    test_docbin.to_disk('test_data.spacy')

100%|██████████| 60/60 [00:01<00:00, 41.86it/s]


In [14]:

import pandas as pd

df = pd.read_json('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json')


In [15]:
df.head()

,0,1
0,Govardhana K Senior Software Engineer Bengalu...,"{'entities': [[1749, 1755, 'Companies worked a..."
1,"Harini Komaravelli Test Analyst at Oracle, Hyd...","{'entities': [[2275, 2281, 'Companies worked a..."
2,Hartej Kathuria Data Analyst Intern - Oracle R...,"{'entities': [[2247, 2573, 'Skills'], [1435, 1..."
3,Ijas Nizamuddin Associate Consultant - State S...,"{'entities': [[4652, 4850, 'Skills'], [4608, 4..."
4,"Imgeeyaul Ansari java developer Pune, Maharas...","{'entities': [[1894, 2173, 'Skills'], [1726, 1..."
